[Source Code Jupyter Notebook](https://github.com/mctosima/belajarpytorch/blob/main/docs/01_HelloWorldnyaDL.ipynb)

# 01. Hello-World Ala Deep Learning

Pada bagian ini kamu diharapkan mampu:
- Memahami _workflow_ training menggunakan PyTorch
- Memahami penggunaan _optimizer_ dan _loss function_
- Memahami proses training sederhana
- Membuat Model Sederhana
- Menggunakan model untuk inferensi
- Mengukur performa model secara sederhana

<div align="center">
<img src="https://images.contentstack.io/v3/assets/blt71da4c740e00faaa/blt3e9883f5dfd008f4/603039d9cb67827268e09219/saltbae_pytorch.jpg" width="350"/>
</div>

> Pada materi kali ini kita akan mencoba membuat model kecil-kecilan untuk memprediksi keluaran dari dua buah masukan berupa boolean. Sederhananya, kita akan membuat model yang mampu memprediksi hasil dari XOR tanpa memberitahu model bahwa operasi ini adalah XOR. Model akan belajar sendiri untuk melakukan operasi XOR.

**Tttt.tapi, saya pun tidak tahu apa itu XOR?**

Baiklah, jadi begini. XOR adalah operasi logika yang menghasilkan nilai `True` jika salah satu dari dua buah masukan bernilai `True` dan yang satunya bernilai `False`. Jika kedua buah masukan bernilai `True` atau `False`, maka hasilnya adalah `False`.

**Masih belum mengerti**

Baiklah, berikut ini adalah tabel logika XOR:

<div align="center">
<img src="https://www.researchgate.net/publication/326006336/figure/tbl1/AS:669038808141849@1536522690995/Truth-table-of-the-logical-operator-XOR.png" width="350"/>
</div>

Misalkan, jika diberi sebuah masukan X1 berupa 1 dan X2 berupa 0, maka hasil dari operasi XOR adalah 1. Jika diberi masukan X1 berupa 0 dan X2 berupa 0, maka hasil dari operasi XOR adalah 0. Begitu juga sebaliknya. ***Sama jadi 0, beda jadi 1.***

---

## Import Library

In [1]:
import torch
import torch.nn as nn

- `import torch` merupakan library utama yang akan kita gunakan. Library ini berisi fungsi-fungsi yang akan kita gunakan untuk membuat model, menghitung loss, dan melakukan training.
- `import torch.nn as nn` merupakan library yang berisi fungsi-fungsi yang akan kita gunakan untuk membuat model.

## Mempersiapkan data masukan

Kita akan membuat dua buah variabel, `x1` dan `x2` dimana masing-masing akan berisi 50 buah nilai boolean. Kita akan menggunakan `torch.rand` untuk menghasilkan nilai random dari 0 sampai 1.

In [2]:
# membuat variabel x berisi nilai boleean berisi 1 atau 0 sebanyak 50 elemen
x1 = torch.randint(0, 2, (50,))
x2 = torch.randint(0, 2, (50,))
print(f"Input X1:\n{x1}")
print(f"Input X2:\n{x2}")

Input X1:
tensor([1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
        1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1])
Input X2:
tensor([1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
        1, 0])


Selanjutnya kita menggabungkan `x1` dan `x2` menjadi satu buah variabel `x` dengan menggunakan `torch.stack`. Karena `x1` dan `x2` masing-masing berukuran `(50,)`, maka `x` akan berukuran `(2, 50)`.

In [3]:
# menggabungkan variabel x1 dan x2 menjadi satu variabel x
x = torch.stack((x1, x2), dim=1)

Setelah itu, kita akan membuat variabel `y` yang berisi nilai boolean dari hasil operasi XOR dari `x1` dan `x2`. Kita akan menggunakan tanda `^` untuk melakukan operasi XOR. Kamu dapat mencocokkan beberapa contoh masukan dan keluaran sebenarnya (`y`). Misalnya pada indeks-0, `x1` bernilai 1 dan `x2` bernilai 1, maka indeks-0 dari `y` bernilai 0.

> **Catatan:** `y` dalam hal ini disebut juga sebagai `label` atau nama lainnya kerap disebut sebagai `ground truth`

In [4]:
# membuat ground-truth dari input x1 dan x2 yang sudah distack pada x
y = x[:, 0] ^ x[:, 1]
print(f"Ground-truth Y:\n{y}")

Ground-truth Y:
tensor([0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1])


Selanjutnya, kita akan membagi dataset `x` dan `y` menjadi `train` dan `test`.

**Mengapa harus dibagi menjadi train dan test?**
- Ketika kita melakukan proses training, label (`y`) akan diberitahu ke model sehingga jawaban sebenarnya akan diketahui model. Namanya juga proses training, tentu model harus tahu jawaban yang benarnya sehingga dia dapat belajar dari kesalahan yang dia buat.
- Oleh karena itu tidak adil jika kita menguji apakah model yang kita buat sudah pinter atau belum dengan bahan belajar yang sudah ada kunci jawabannya
- Maka dari itu kita menyediakan test data sebagai bahan ujian untuk mengetahui seberapa baik model yang kita buat.

**Lalu apa bedanya dengan validation?**
- Validation adalah data yang digunakan untuk mengetahui seberapa baik model kita dalam melakukan generalisasi. Jika model kita sudah pinter, maka model akan dapat memprediksi dengan baik pada data yang belum pernah dilihat sebelumnya.
- Validation kita gunakan pada saat proses training, sedangkan test kita gunakan pada saat proses inferensi akhir untuk menguji seberapa baik model kita.

> **Analogi Sederhana:**
> Train adalah bahan belajar yang sudah diketahui jawabannya. Mirip seperti soal-soal yang ada di buku pelajaran.
> Validation adalah bahan belajar yang belum diketahui jawabannya. Mirip seperti soal-soal yang ada di ujian tengah semester. Tujuannya untuk mengukur apakah kita sudah cukup baik dalam mempelajari materi atau belum. Jika hasil ujian kita kurang baik, maka kita dapat belajar lagi.
> Test adalah ujian akhir, mirip seperti ujian akhir nasional. Tujuannya mengukur seberapa baik kita dalam belajar dibandingkan dengan siswa lainnya.

**Bagaimana biasanya pembagian antara train, val, dan test**

| **Pembagian** | **Tujuan**            | **Persentase**    |
| ---           | ---                   | ---               |
| Train         | Sebagai bahan belajar | 70% - 80%         |
| Validation    | Mengukur generalisasi | 10% - 20%         |
| Test          | Mengukur akurasi      | 10% - 20%         |


In [5]:
train_split = int(0.8*len(x))

In [6]:
# membagi x menjadi train dan testing
x_train = x[:train_split]
x_test = x[train_split:]

# membagi y menjadi train dan testing
y_train = y[:train_split]
y_test = y[train_split:]

**Penjelasan**
- Dalam contoh di atas, kita membagi dataset menjadi 80% train dan 20% test
- Agar pembagiannya bulat, kita menggunakan `int()` untuk membulatkan hasil pembagian

## Membuat Model
Untuk membuat model menggunakan framework PyTorch, dilakukan dengan cara menggunakan kelas `nn.Module`. Kelas ini akan menjadi kelas induk dari model yang kita buat. Kelas ini akan berisi fungsi-fungsi yang akan kita gunakan untuk membuat model. Dalam contoh sederhana ini terdapat dua buah fungsi, yaitu `__init__` dan `forward`.

### Fungsi `__init__`
Fungsi ini akan dijalankan ketika sebuah instance dibuat. Dalam hal ini, ketika sebuah instance model di buat, maka akan terbentuk beberapa buah layer yang akan kita gunakan untuk membuat model.

#### <a name="nnlinear"></a>`nn.Linear`
Layer pertama yang akan dibuat adalah `self.linear = nn.Linear(2,2)` merupakan sebuah layer regresi linear yang memiliki 2 buah input dan 2 buah output. Ilustrasinya dapat dilihat pada gambar di bawah ini

<img src="https://www.sharetechnote.com/image/Python_Pytorch_nn_Linear_i2_o2_01.png" width=350>

_Sumber: ShareTechnote_

Layer ini akan menerima masukan `x1` dan `x2` yang sudah kita buat sebelumnya. Dan masing-masing input akan dihubungkan dengan dua buah Neuron. Setiap hubungan yang ada akan memiliki sebuah bobot (disebut sebagai **weight**, simbol w). Selain itu setiap neuron akan memiliki sebuah bias (simbol b). Dalam deep learning, weight dan bias akan diubah-ubah secara otomatis oleh model sehingga model dapat belajar dari data yang diberikan.

Selain `nn.Linear(2,2)` kita juga membuat layer `self.linear2 = nn.Linear(2,1)`. Layer ini akan menerima masukan dari layer sebelumnya yaitu `self.linear`. Dan layer ini akan menghasilkan satu buah output.

> Ketika diinisiasi secara otomatis oleh PyTorch, Weight dan Bias akan diinisiasi secara random. Jika kita ingin menginisiasi secara manual, kita dapat menggunakan parameter `weight` dan `bias` pada fungsi `nn.Linear`

#### Fungsi Sigmoid

Fungsi sigmoid adalah sebuah fungsi yang mengubah nilai masukan menjadi rentang antara 0 dan 1. Fungsi ini digunakan karena nilai keluaran yang kita butuhkan dalam kasus prediksi ini haruslah pada rentang 0 dan 1 karena masukan kita juga menggunakan tipe data boolean dan kita mengharapkan keluaran dalam bentuk boolean juga nantinya. Ilustrasinya dapat dilihat pada gambar di bawah ini

<img src ="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/1200px-Logistic-curve.svg.png" width=350>

[_Sumber: Wikipedia_](https://en.wikipedia.org/wiki/Sigmoid_function)

### Fungsi `forward`

Fungsi forward (atau method Forward) pada PyTorch merupakan fungsi yang akan dijalankan ketika kita melakukan proses forward propagation. Secara sederhana, forward propagation dapat diibaratkan ***proses dan tahapan yang dilakukan terhadap input (`x`) hingga ia menghasilkan output (`y`)***.

In [7]:
# membuat model regresi linear
class XORModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 2)
        self.sigmoid = nn.Sigmoid()
        self.linear2 = nn.Linear(2, 1)
        

    def forward(self, x):
        x = x.float()
        x = self.linear(x)
        x = self.sigmoid(x)
        x = self.linear2(x)
        x = self.sigmoid(x)
        return x

**Penjelasan `Class XORModel`**

- `self.linear` merupakan layer yang akan menerima masukan `x1` dan `x2` dan menghasilkan dua buah output
- `self.linear2` merupakan layer yang akan menerima masukan dari layer sebelumnya yaitu `self.linear` dan menghasilkan satu buah output
- `self.sigmoid` digunakan untuk mengubah nilai keluaran dari layer `self.linear` dan `self.linear2` menjadi rentang antara 0 dan 1
- pada fungsi `forward` data masukan x terlebih dahulu diubah dalam bentuk float
- kemudian data masukan x dihubungkan dengan layer `self.linear` dan `self.linear2` menggunakan fungsi `self.sigmoid`

Setelah kelas model terbentuk, kita akan membuat instance dari kelas model tersebut. Instance ini akan kita sebut sebagai `model`.

In [8]:
# create instance
model = XORModel()

# check state_dict
print(model.state_dict())

OrderedDict([('linear.weight', tensor([[ 0.5084, -0.6865],
        [ 0.0465,  0.3471]])), ('linear.bias', tensor([0.1432, 0.3717])), ('linear2.weight', tensor([[-0.2189,  0.5219]])), ('linear2.bias', tensor([0.5518]))])


## Mencoba Prediksi Tanpa Training

Sebelum melakukan training, mari kita coba model yang telah kita buat dengan data testing (`x_test`). Untuk melakukan prediksi, caranya sangat mudah, kita hanya perlu melakukan forward propagation dengan memanggil fungsi `model.forward(x_test)`.

**Mengapa menggunakan `with torch.no_grad()`?**
`with torch.no_grad()` digunakan untuk menghindari proses gradient descent. Karena kita hanya ingin melakukan prediksi tanpa melakukan training, maka kita tidak perlu melakukan proses gradient descent.

**Apa itu gradient descent?**
Gradient descent adalah proses yang dilakukan oleh model untuk melakukan perubahan pada bobot dan bias. Proses ini dilakukan secara otomatis oleh model sehingga kita tidak perlu melakukan perubahan secara manual.

> Intinya, `with torch.no_grad()` kita lakukan ketika kita hanya ingin si model melakukan prediksi tanpa harus belajar dari data yang diberikan.

In [9]:
# mencoba inferensi dengan input x
with torch.no_grad():
    y_pred = model(x_test)
print(f"Input:\n{x_test}")
print(f"Hasil prediksi:\n{y_pred}")
print(f"Ground-truth:\n{y_test}")

Input:
tensor([[0, 1],
        [1, 0],
        [1, 1],
        [0, 0],
        [0, 1],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 1],
        [1, 0]])
Hasil prediksi:
tensor([[0.6947],
        [0.6732],
        [0.6901],
        [0.6778],
        [0.6947],
        [0.6778],
        [0.6778],
        [0.6778],
        [0.6947],
        [0.6732]])
Ground-truth:
tensor([1, 1, 0, 0, 1, 0, 0, 0, 1, 1])


Bagaimana hasil prediksinya? Tentu saja hancur. Jelas saja, mustahil model dapat memprediksi tanpa melakukan training terlebih dahulu. Kita akan melakukan training pada model kita pada bagian selanjutnya. Namun sebelum itu, mari kita mempersiapkan `loss function` dan `optimizer` yang akan kita gunakan.

## Melakukan Training

### Mengatur Loss Function dan Optimizer

**Apa itu loss function?**
Loss function adalah sebuah fungsi yang digunakan untuk mengukur seberapa baik model kita dalam melakukan prediksi. Semakin kecil nilai loss function, maka semakin baik model kita dalam melakukan prediksi. Loss function memberitahu model "seberapa parah" kesalahan prediksi yang ia lakukan. Sehingga di epoch-epoch selanjutnya, model dapat melakukan perbaikan terhadap kesalahan yang ia lakukan pada epoch sebelumnya.

**Apa itu optimizer?**
Setelah model tahu seberapa parah kesalahannya, maka model harus belajar. Optimizerlah yang akan melakukan proses belajar model. Optimizer akan melakukan perubahan pada bobot (weight) dan bias (b) model agar model dapat melakukan prediksi yang lebih baik pada epoch selanjutnya. Lihat gambar [di bagian sebelumnya](#nnlinear) jika sudah lupa terkait weight dan bias.



In [10]:
# mendefinisikan loss function
loss_fn = nn.MSELoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

# epoch
epochs = 5000

**Penjelasan**

Pada contoh diatas kita menggunakan loss function `nn.MSELoss()` dan optimizer `torch.optim.SGD(model.parameters(), lr=0.01)`. Loss function yang digunakan adalah `nn.MSELoss()` yang merupakan loss function yang paling sederhana. Loss function ini akan menghitung selisih antara nilai prediksi dan nilai aktual. Kemudian nilai selisih tersebut akan dipangkatkan dua dan dijumlahkan. Semakin kecil nilai loss function, maka semakin baik model kita dalam melakukan prediksi.

Equationnya dapat kamu temukan pada [tautan berikut ini](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html)

**Contoh Perhitungan Loss Function `nn.MSELoss()`**

Misalkan kita memiliki `x1 = 1` dan `x2 = 0`. Nilai aktual dari `x1` dan `x2` adalah `y = 1`. Nilai prediksi dari `x1` dan `x2` adalah `y_pred = 0.8`. Dengan menggunakan MSELoss kita dapat menghitung nilai loss function dengan cara berikut:
- `loss = (y - y_pred) ** 2`
- `loss = (1 - 0.8) ** 2`
- `loss = 0.04`

**Tentang Optimizer SGD**

Optimizer SGD adalah salah satu optimizer yang paling sederhana. Optimizer ini akan melakukan perubahan pada bobot dan bias model dengan cara mengurangi nilai bobot dan bias dengan nilai gradient. Nilai gradient ini dihitung dengan cara mengalikan nilai learning rate dengan nilai gradient dari loss function. Semakin kecil nilai loss function, maka semakin kecil nilai gradient. Semakin kecil nilai gradient, maka semakin kecil nilai perubahan yang dilakukan pada bobot dan bias.

> **Catatan**
> Jika hingga tahap ini kamu merasa bingung dengan Loss Function dan Optimizer, itu wajar. Semoga di modul-modul selanjutnya kamu akan semakin paham dengan Loss Function dan Optimizer.

### Melakukan Training

Setelah kita menyiapkan loss function dan optimizer, sekarang saatnya kita melakukan training. Training dilakukan dengan cara melakukan forward propagation dan backward propagation pada model. Forward propagation dilakukan dengan memanggil fungsi `model.forward(x_train)` dan backward propagation dilakukan dengan memanggil fungsi `loss.backward()`.

Tentunya training akan dilakukan berulang kali. Banyaknya pengulangan training ini disebut sebagai `epoch`. Pada contoh diatas, kita melakukan training sebanyak 1000 epoch. Setiap epoch, kita akan melakukan forward propagation dan backward propagation. Kemudian kita akan menghitung nilai loss function dan menampilkan nilai loss function tersebut.

In [11]:
# training loop
for epoch in range(epochs):
    # training
    y_pred = model(x_train)
    y_train = y_train.float()
    loss = loss_fn(y_pred, y_train.unsqueeze(1))

    # backward
    optimizer.zero_grad()
    
    loss.backward()
    
    
    optimizer.step()
    
    # testing
    with torch.no_grad():
        y_test_pred = model(x_test)
        y_test = y_test.float()
        test_loss = loss_fn(y_test_pred, y_test.unsqueeze(1))

    # print loss only 3 digit after comma. Log every 500 epoch
    if (epoch+1) % 200 == 0:
        print(f"Epoch: {epoch+1}, Loss: {loss.item():.3f}, Test Loss: {test_loss.item():.3f}")

Epoch: 200, Loss: 0.221, Test Loss: 0.245
Epoch: 400, Loss: 0.162, Test Loss: 0.189
Epoch: 600, Loss: 0.117, Test Loss: 0.146
Epoch: 800, Loss: 0.031, Test Loss: 0.039
Epoch: 1000, Loss: 0.011, Test Loss: 0.014
Epoch: 1200, Loss: 0.007, Test Loss: 0.008
Epoch: 1400, Loss: 0.005, Test Loss: 0.005
Epoch: 1600, Loss: 0.003, Test Loss: 0.004
Epoch: 1800, Loss: 0.003, Test Loss: 0.003
Epoch: 2000, Loss: 0.002, Test Loss: 0.003
Epoch: 2200, Loss: 0.002, Test Loss: 0.002
Epoch: 2400, Loss: 0.002, Test Loss: 0.002
Epoch: 2600, Loss: 0.002, Test Loss: 0.002
Epoch: 2800, Loss: 0.001, Test Loss: 0.002
Epoch: 3000, Loss: 0.001, Test Loss: 0.001
Epoch: 3200, Loss: 0.001, Test Loss: 0.001
Epoch: 3400, Loss: 0.001, Test Loss: 0.001
Epoch: 3600, Loss: 0.001, Test Loss: 0.001
Epoch: 3800, Loss: 0.001, Test Loss: 0.001
Epoch: 4000, Loss: 0.001, Test Loss: 0.001
Epoch: 4200, Loss: 0.001, Test Loss: 0.001
Epoch: 4400, Loss: 0.001, Test Loss: 0.001
Epoch: 4600, Loss: 0.001, Test Loss: 0.001
Epoch: 4800, Lo

**Penjelasan Tahap Training**
- `for epoch in range(epochs)` maksudnya adalah kita melakukan training sebanyak 1000 epoch. Variabel epochs sudah didefinisikan di bagian sebelumnya
- `y_pred = model(x_train)` adalah proses forward propagation. Pada tahap ini, model akan melakukan prediksi terhadap nilai x_train yang diberikan. Nilai prediksi tersebut akan disimpan pada variabel `y_pred`. Selanjutnya pada program diatas, nilai `y_pred` kita ubah menjadi float untuk dapat dihitung lossnya.
- `loss = loss_fn(y_pred, y_train.unsqueeze(1))` adalah proses menghitung nilai loss function. Nilai loss function ini akan disimpan pada variabel `loss`. Kita melakukan unsqueeze terhadap `y_train` karena nilai `y_train` yang kita miliki adalah 1 dimensi. Sedangkan nilai `y_pred` yang kita miliki adalah 2 dimensi. Kita perlu melakukan unsqueeze agar nilai `y_train` dan `y_pred` memiliki dimensi yang sama.
- `optimizer.zero_grad()` adalah proses mengosongkan nilai gradient yang ada pada model. Karena pada setiap epoch, nilai gradient akan berubah. Jika nilai gradient tidak direset, maka nilai gradient pada epoch selanjutnya akan dihitung berdasarkan nilai gradient pada epoch sebelumnya.
- `loss.backward()` adalah proses backward propagation. Pada tahap ini, model akan melakukan perubahan pada bobot dan biasnya.
- `optimizer.step()` adalah proses mengupdate nilai bobot dan bias model. Nilai bobot dan bias model akan diupdate berdasarkan nilai gradient yang ada pada model.
- `with torch.no_grad():` adalah proses untuk memvalidasi. Perbedaan validasi dan proses training adalah pada proses validasi, kita tidak melakukan perubahan pada bobot dan bias model. Kita hanya melakukan forward propagation saja. Kita menggunakan `with torch.no_grad():` agar model tidak melakukan perubahan pada bobot dan biasnya. Isi dari bagian ini sama dengan bagian training sebelumnya.
- `if (epoch+1) % 200 == 0:` adalah proses untuk menampilkan nilai loss function. Kita akan menampilkan nilai loss function setiap 200 epoch. Kita menggunakan `if (epoch+1) % 200 == 0:` agar kita tidak perlu menampilkan nilai loss function setiap epoch.

**Hasil Proses Training**
Dapat dilihat pada hasil proses training, pada epoch ke 800 nilai loss sudah sangat kecil. Artinya, model sudah cukup baik dalam melakukan prediksi terhadap nilai x_train yang diberikan. Dapat disimpulkan model berhasil belajar.

## Evaluasi Model

Selanjutnya kita melakukan proses testing kecil-kecilan. Model yang sebelumnya sudah dilatih akan mencoba untuk memprediksi hasil dari nilai x yang kita berikan, dalam hal ini disimpan dalam variabel `x_user`. Kita akan memasukkan nilai `x_user` ke dalam model dan model akan melakukan prediksi terhadap nilai `x_user`. Hasil prediksi tersebut akan disimpan dalam variabel `y_user`. Kemudian kita akan menampilkan hasil prediksi tersebut.

In [12]:
# mencoba inferensi dengan input yang diberikan pengguna
x_user = torch.tensor([[1, 0]])
y_truth = x_user[:, 0] ^ x_user[:, 1]
y_pred = model(x_user)

print(f"Input:\n{x_user}")
print(f"Hasil prediksi:\n{y_pred}")
print(f"Ground-truth:\n{y_truth}")


Input:
tensor([[1, 0]])
Hasil prediksi:
tensor([[0.9707]], grad_fn=<SigmoidBackward0>)
Ground-truth:
tensor([1])


## Kesimpulan

<img src ="https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/01_a_pytorch_workflow.png" width = 700>

Keseluruhan dari modul ini pada dasarnya ingin memberikan gambaran kepada kamu tentang 'cara main' di PyTorch. Workflow sederhana untuk membuat model machine learning dengan PyTorch adalah sebagai berikut:
1. Mempersiapkan data - mempersiapkan data untuk digunakan dalam model machine learning. Bisa jadi memuat data dari databse, dari teks, gambar, dsb. Atau bisa juga membuat data sendiri.
2. Menyiapkan model - dapat berupa membuat model dari awal, atau memuat model yang sudah ada.
    
    - Menyiapkan loss function dan optimizer - loss function digunakan untuk mengukur seberapa baik model kita dalam melakukan prediksi. Optimizer digunakan untuk mengupdate nilai bobot dan bias model.
    
    - Menyiapkan training loop - training loop digunakan untuk melakukan training model. Training loop ini akan melakukan forward propagation dan backward propagation pada model. Kemudian akan menghitung nilai loss function dan mengupdate nilai bobot dan bias model.
3. Training - melakukan training model dengan menggunakan data yang sudah kita miliki, dengan menggunakan loss function dan optimizer yang sudah kita siapkan dan dengan menggunakan training loop yang sudah kita siapkan.
4. Evaluasi - melakukan evaluasi model dengan menggunakan data yang belum pernah dilihat oleh model sebelumnya. Dengan melakukan evaluasi ini, kita bisa mengetahui seberapa baik model kita dalam melakukan prediksi. Jika hasilnya kurang baik, kita bisa melakukan tuning pada model, loss function, optimizer, dsb (mundur ke langkah-langhkah sebelumnya).
5. Perbaikan - melakukan perbaikan pada model, loss function, optimizer, dsb. Kemudian kembali ke langkah training untuk melihat apakah perbaikan yang kita lakukan sudah berhasil atau belum.
6. Deploy - model yang sudah kita buat sudah siap untuk digunakan. Model ini bisa kita gunakan untuk memprediksi data baru yang belum pernah dilihat oleh model sebelumnya. Di tahap inilah model machine learning kita akan berinteraksi dengan dunia nyata.

---
# Latihan Modul 01

Jawaban dapat dilihat pada tautan [berikut ini](https://github.com/mctosima/belajarpytorch/blob/main/solution/01_Solution.ipynb)

1. Buatlah sebuah data masukan berupa variabel x1, x2, dan x3, masing-masing sebanyak 100 elemen. Data tersebut berupa nilai integer 1 atau 0.
2. Gabungkanlah data masukan dari x1, x2, dan x3 menggunakan fungsi `torch.stack()`. Ubahlah x menjadi tipe data float
3. Buatlah sebuah _ground truth_ untuk operasi dari x1 XOR x2 NAND x3. Gunakan fungsi `torch.logical_xor()` dan `torch.logical_not()`. Konversikan hasilnya menjadi float
4. Lakukan split terhadap x dan y dengan persentase 80% untuk data training dan 20% untuk data testing.
5. Buatlah model sederhana yang sama dengan model yang dibahas pada modul 01. Gunakan fungsi `nn.Linear()` untuk membuat model tersebut. Kamu dapat melakukan copy paste dari modul 01 namun jangan lupa untuk mengubah input dan output layer menjadi 3 yang tadinya 2
6. Muat kelas model yang dibuat pada variabel `model` lalu gunakan fungsi `nn.CrossEntropyLoss()` untuk membuat fungsi loss dan `torch.optim.Adam()` untuk membuat optimizer dengan learning rate sebesar 0.1. 
7. Buatlah training loop dengan epoch sebanyak 2000. Cetak loss setiap 100 epoch.
8. Dengan menggunakan masukan berupa `[0, 0, 1]` dan `[1, 1, 0]` lakukan prediksi dengan model yang telah dibuat. Apakah hasilnya sudah tepat?

Jika anda mengalami kesulitan dalam pengerjaan, silahkan tanyakan melalui [forum diskusi berikut ini](https://github.com/mctosima/belajarpytorch/discussions), kami akan segera menjawabnya.

# Sumber Belajar Tambahan
1. [Youtube Video: Gradient Descent](https://www.youtube.com/watch?v=IHZwWFHWa-w) by 3Blue1Brown
2. [Youtube Video: Back Propagation](https://www.youtube.com/watch?v=Ilg3gGewQ5U&feature=youtu.be) by 3Blue1Brown